In [1]:
import os
import cv2
import json
import numpy as np
import time

# 從自訂模組引入 FFmpeg 讀取/寫入工具
from utils.video_utils import ReadArray, WriteArray

# 檢查工作目錄，方便定位檔案
print('Current working directory:', os.getcwd())

Current working directory: d:\works\Hsu_Lab\Animal_Visusal_Fundation_Model\MotiClip\test\20250317


In [2]:
# === Step 1: 使用 FFmpegVideoReader 讀取影片 ===
VIDEO_PATH = 'data/video.mp4'  # 測試影片檔案路徑
reader = ReadArray(VIDEO_PATH)
print(f"影片資訊: 寬度={reader.width}, 高度={reader.height}, FPS={reader.fps}, 總影格數={len(reader)}")

影片資訊: 寬度=1440, 高度=1080, FPS=25.0, 總影格數=41818


In [3]:
# === Step 2: 從 JSON 讀取 Arena 座標 ===
JSON_PATH = 'data/video_info.json'  
with open(JSON_PATH, 'r') as f:
    metadata = json.load(f)

# 根據範例，arena 資訊存於 'arena' 欄位，每個 arena 為一個字典，包含 id 與 points
arenas = metadata.get('arena', [])
print(f"找到 {len(arenas)} 個 Arena")

# 假設輸出影像的統一解析度為 480x480
OUTPUT_SIZE = (720, 720)  
# 定義目的座標點（左上、右上、右下、左下）
dst_pts = np.array([
    [0, 0],
    [OUTPUT_SIZE[0], 0],
    [OUTPUT_SIZE[0], OUTPUT_SIZE[1]],
    [0, OUTPUT_SIZE[1]]
], dtype=np.float32)

print('目的點座標:', dst_pts)

找到 4 個 Arena
目的點座標: [[  0.   0.]
 [720.   0.]
 [720. 720.]
 [  0. 720.]]


In [4]:
import sys
import numpy as np
import cv2
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

def process_arena(arena, position):
    """
    處理單個 arena：
      - 建立自己的讀取器、計算透視變換矩陣
      - 初始化影片寫入器
      - 逐影格進行讀取、透視變換、寫入
      - 更新進度條 (tqdm)
    """
    # 每個 arena 都重新初始化讀取器
    reader = ReadArray(VIDEO_PATH)
    arena_id = arena.get('id', 'arena_unknown')
    src_pts = np.array(arena.get('points'), dtype=np.float32)
    print(f"處理 {arena_id}，來源點座標: {src_pts}", flush=True)
    
    # 計算透視變換矩陣
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    
    # 輸出檔案名稱 (將存於腳本同目錄)
    output_video = f"{arena_id}_output.mp4"
    
    # 初始化影片寫入器
    writer = WriteArray(output_video, reader.fps)
    
    total_frames = len(reader)
    # 建立進度條，position 參數用來區分各個 arena 的進度條顯示位置
    pbar = tqdm(total=total_frames, desc=f"Arena {arena_id}", position=position, leave=True)
    
    for frame_index in range(total_frames):
        # 讀取原始影格
        frame = reader[frame_index]
        if frame is None:
            print(f"警告: {arena_id} 無法讀取 frame {frame_index}，中止處理。", flush=True)
            break
        
        # 透視變換
        warped_frame = cv2.warpPerspective(frame, M, OUTPUT_SIZE, flags=cv2.INTER_LINEAR)
        # 寫入統一格式影片
        writer.append(warped_frame)
        pbar.update(1)
    
    writer.close()
    pbar.close()
    return f"{arena_id}: 輸出完成，影片儲存於 {output_video}"

results = []
# 使用 ThreadPoolExecutor 同時處理所有 arena，max_workers 設為 arena 數量
with ThreadPoolExecutor(max_workers=len(arenas)) as executor:
    futures = []
    for pos, arena in enumerate(arenas):
        futures.append(executor.submit(process_arena, arena, pos))
    for future in futures:
        results.append(future.result())

# 列印每個 arena 的處理結果
for res in results:
    print(res)


處理 arena_3，來源點座標: [[ 378.66666  567.99994]
 [ 709.3333   565.3333 ]
 [ 717.3333  1013.3333 ]
 [ 381.33334 1013.3333 ]]
處理 arena_4，來源點座標: [[ 728.       554.6666 ]
 [1072.       549.3333 ]
 [1077.3334  1002.6666 ]
 [ 733.3333  1007.99994]]
處理 arena_1，來源點座標: [[378.66666  82.66667]
 [709.3333   80.00001]
 [709.3333  541.3333 ]
 [378.66666 541.3333 ]]
處理 arena_2，來源點座標: [[ 728.        74.66667]
 [1069.3334    72.00001]
 [1077.3334   528.     ]
 [ 733.3333   533.3333 ]]


Arena arena_2:   0%|          | 0/41818 [00:00<?, ?it/s]

Arena arena_1:   0%|          | 0/41818 [00:00<?, ?it/s]

Arena arena_4:   0%|          | 0/41818 [00:00<?, ?it/s]

Arena arena_3:   0%|          | 0/41818 [00:00<?, ?it/s]

arena_1: 輸出完成，影片儲存於 arena_1_output.mp4
arena_2: 輸出完成，影片儲存於 arena_2_output.mp4
arena_3: 輸出完成，影片儲存於 arena_3_output.mp4
arena_4: 輸出完成，影片儲存於 arena_4_output.mp4
